In [10]:
from models import VCLModel, VanillaMLP
from models.kcoreset import KClusteringCoreset
from datasets import PermutedMnist

import matplotlib.pyplot as plt
import numpy as np

import os

import torch
import torch.nn.functional as F

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
NUM_EPOCHS = 50
LR = 1e-3
BATCH_SIZE = 256
TRAIN_NUM_SAMPLES = 10
TEST_NUM_SAMPLES = 100
CORESET_SIZE = 200
INIT_MLP = False

MLP_INIT_DESC = 'withoutmlpinit' if not INIT_MLP else 'withmlpinit'

random_seed = 1
torch.manual_seed(random_seed + 1)
np.random.seed(random_seed)

model_base_name = 'vcl_clusters_{}_lr_{}_{}_batch_{}_coresetsize_{}_epochs_{}_run_{}_no_tasks_{}'

In [15]:
no_tasks = 10

dataset = PermutedMnist(no_tasks)

In [16]:
NO_RUNS = 1

train_loaders, test_loaders = dataset.get_tasks(batch_size=BATCH_SIZE)
no_tasks = dataset.no_tasks

# Modify as need be.
epochs = [50]
coreset_sizes = [1000, 5000]
methods = ['kmeans']

for method in methods:
    METHOD = method
    
    clustering_coreset_selector = KClusteringCoreset(dataset)
    coreset_loaders, sans_coreset_loaders = clustering_coreset_selector.get_coreset_loaders(batch_size=BATCH_SIZE, coreset_size=CORESET_SIZE, method=METHOD) 
    
    for coreset_size in coreset_sizes:
        CORESET_SIZE = coreset_size
        
        for epoch in epochs:
            NUM_EPOCHS = epoch
            
            for run_number in range(NO_RUNS):
                model_name = model_base_name.format(METHOD, LR, MLP_INIT_DESC, BATCH_SIZE, CORESET_SIZE, NUM_EPOCHS, run_number, no_tasks) 
               
                LOG_DIR = 'logs/k_clusters/'
                os.makedirs(LOG_DIR, exist_ok=True)
                
                LOG_FILE_NAME = os.path.join(LOG_DIR, model_name) + '.txt'
                log_file_handler = open(LOG_FILE_NAME, "w")
                print("Run Number: {}. Log File: {}".format(run_number, LOG_FILE_NAME))
                
                train_loaders = sans_coreset_loaders
                
                model = VCLModel(784, 10, [100, 100])
                accuracies = np.zeros((no_tasks, no_tasks))
                
                for i in range(no_tasks):
                    train_loader = train_loaders[i]
                    
                    if INIT_MLP and i == 0:
                        print("Training MLP model to init first task")
                        mlp = VanillaMLP(784, 10, [100, 100])
                        mlp.train_model(NUM_EPOCHS, train_loader, LR)
                        model.init_mle(mlp)
                    
                    model.train_model(NUM_EPOCHS, train_loader, LR, TRAIN_NUM_SAMPLES)
                    
                    model.update_priors()
                    
                    # Train on coreset after training on non-coreset.
                    if CORESET_SIZE > 0:
                        coreset_loader = coreset_loaders[i]
                        model.train_model(NUM_EPOCHS, coreset_loader, LR, TRAIN_NUM_SAMPLES)
                        
                    task_accs = []
                    for j in range(i + 1):
                        test_loader = test_loaders[j]
                        accuracy = model.get_accuracy(test_loader, 100)
                        
                        msg = "[Number Tasks: {}]: Task {} Accuracy: {}".format(j + 1, j + 1, accuracy ) 
                        log_file_handler.write(msg + '\n')
                        log_file_handler.flush()
                        print(msg)
                        
                        task_accs.append(accuracy)
                        accuracies[i][j] = accuracy
                    
                    msg = "Task {} / {}. Mean Accuracy: {}".format(i + 1, no_tasks, np.mean(task_accs))
                    log_file_handler.write(msg + '\n')
                    log_file_handler.flush()
                    print(msg)
                log_file_handler.close()

  0%|          | 0/10 [00:00<?, ?it/s]

Processing class: 0
Processing class: 1
Processing class: 2
Processing class: 3
Processing class: 4
Processing class: 5
Processing class: 6
Processing class: 7
Processing class: 8
Processing class: 9


 10%|█         | 1/10 [00:42<06:26, 42.95s/it]

Processing class: 0
Processing class: 1
Processing class: 2
Processing class: 3
Processing class: 4
Processing class: 5
Processing class: 6
Processing class: 7
Processing class: 8
Processing class: 9


 20%|██        | 2/10 [01:16<04:57, 37.21s/it]

...

[Number Tasks: 1]: Task 1 Accuracy: 0.9515
[Number Tasks: 2]: Task 2 Accuracy: 0.9347
[Number Tasks: 3]: Task 3 Accuracy: 0.9404
[Number Tasks: 4]: Task 4 Accuracy: 0.9433
[Number Tasks: 5]: Task 5 Accuracy: 0.9518
[Number Tasks: 6]: Task 6 Accuracy: 0.9539
[Number Tasks: 7]: Task 7 Accuracy: 0.958
[Number Tasks: 8]: Task 8 Accuracy: 0.9586
Task 8 / 10. Mean Accuracy: 0.949025


Total Loss: 0.3108655105416591, KL: 0.13719172329984158, Lik Loss: 0.17367378717813736: 100%|██████████| 50/50 [05:05<00:00,  6.12s/it] 
Total Loss: 0.023140120087191463, KL: 0.011127818375825882, Lik Loss: 0.012012301594950259: 100%|██████████| 50/50 [00:23<00:00,  2.13it/s]


[Number Tasks: 1]: Task 1 Accuracy: 0.9488
[Number Tasks: 2]: Task 2 Accuracy: 0.9263
[Number Tasks: 3]: Task 3 Accuracy: 0.9353
[Number Tasks: 4]: Task 4 Accuracy: 0.9394
[Number Tasks: 5]: Task 5 Accuracy: 0.947
[Number Tasks: 6]: Task 6 Accuracy: 0.9502
[Number Tasks: 7]: Task 7 Accuracy: 0.9512
[Number Tasks: 8]: Task 8 Accuracy: 0.9556
[Number Tasks: 9]: Task 9 Accuracy: 0.9575
Task 9 / 10. Mean Accuracy: 0.9457


Total Loss: 0.30728894841467214, KL: 0.1350776156426495, Lik Loss: 0.1722113328675429: 100%|██████████| 50/50 [04:47<00:00,  5.75s/it]  
Total Loss: 0.026060479460284114, KL: 0.013166406308300793, Lik Loss: 0.01289407315198332: 100%|██████████| 50/50 [00:21<00:00,  2.36it/s] 


[Number Tasks: 1]: Task 1 Accuracy: 0.9475
[Number Tasks: 2]: Task 2 Accuracy: 0.9231
[Number Tasks: 3]: Task 3 Accuracy: 0.9332
[Number Tasks: 4]: Task 4 Accuracy: 0.9364
[Number Tasks: 5]: Task 5 Accuracy: 0.9465
[Number Tasks: 6]: Task 6 Accuracy: 0.9468
[Number Tasks: 7]: Task 7 Accuracy: 0.9487
[Number Tasks: 8]: Task 8 Accuracy: 0.9508
[Number Tasks: 9]: Task 9 Accuracy: 0.9547
[Number Tasks: 10]: Task 10 Accuracy: 0.9585
Task 10 / 10. Mean Accuracy: 0.9446200000000001


In [4]:
kcc =KClusteringCoreset(dataset)
kcc.get_coreset_datasets(200, method='kcenter_greedy')

Computing Cluster Coresets...:  20%|██        | 2/10 [00:49<03:16, 24.59s/it]


KeyboardInterrupt: 